# Comparraison de groupe d'image

In [2]:
import numpy as np
import PIL.Image as Image
import os

In [3]:
def render_comp(im_group1, im_group2):
    import numpy as np

    if len(im_group1) != len(im_group2):
        raise ValueError("Image groups must have the same number of images.")

    total_diff = []
    num_images = len(im_group1)

    for im1, im2 in zip(im_group1, im_group2):
        diff = comp_2_img(im1, im2)
        total_diff.append(diff)
        print(f"Compared image {len(total_diff)}/{num_images}")

    return mean_diff(total_diff)



def comp_2_img(im1, im2, use_alpha_weight=True):

    # ---------------------------
    # 1. Sanity checks
    # ---------------------------
    if im1.shape != im2.shape:
        raise ValueError("Input images must have the same dimensions.")
    if im1.shape[2] != 4:
        raise ValueError("Images must be in RGBA format (4 channels).")

    # ---------------------------
    # 2. Extract channels
    # ---------------------------
    im1_rgb = im1[..., :3].astype(np.float32)
    im2_rgb = im2[..., :3].astype(np.float32)

    alpha1 = im1[..., 3].astype(np.float32) / 255.0
    alpha2 = im2[..., 3].astype(np.float32) / 255.0

    # Common visible area
    mask = (alpha1 > 0) & (alpha2 > 0)

    if np.count_nonzero(mask) == 0:
        raise ValueError("No overlapping visible pixels between images.")

    # Alpha weighting (anti-aliasing friendly)
    if use_alpha_weight:
        alpha = np.minimum(alpha1, alpha2)
    else:
        alpha = mask.astype(np.float32)

    alpha = alpha[..., None]  # broadcast to RGB

    # ---------------------------
    # 3. Error-based metrics
    # ---------------------------
    diff_dict = {}

    abs_diff = np.abs(im1_rgb - im2_rgb) * alpha
    sq_diff = ((im1_rgb - im2_rgb) ** 2) * alpha

    diff_dict["NMAE"] = abs_diff.sum() / (alpha.sum() * 255.0)
    diff_dict["NMSE"] = sq_diff.sum() / (alpha.sum() * (255.0 ** 2))

    # ---------------------------
    # 4. ZNCC (global structure)
    # ---------------------------
    im1_vec = im1_rgb[mask].reshape(-1, 3)
    im2_vec = im2_rgb[mask].reshape(-1, 3)

    im1_mean = im1_vec - im1_vec.mean(axis=0)
    im2_mean = im2_vec - im2_vec.mean(axis=0)

    numerator = np.sum(im1_mean * im2_mean)
    denominator = np.sqrt(
        np.sum(im1_mean ** 2) * np.sum(im2_mean ** 2)
    )

    diff_dict["ZNCC"] = numerator / denominator if denominator > 0 else 0.0

    # ---------------------------
    # 5. PSNR (quality metric)
    # ---------------------------
    mse = sq_diff.sum() / alpha.sum()
    diff_dict["PSNR"] = (
        10 * np.log10((255.0 ** 2) / mse) if mse > 0 else np.inf
    )

    return diff_dict



def mean_diff(diff_list):
    mean_diff = dict()
    num_diffs = len(diff_list)

    if num_diffs == 0:
        return mean_diff

    # Initialize sums
    for key in diff_list[0].keys():
        mean_diff[key] = 0.0

    # Sum up all metrics
    for diff in diff_list:
        for key, value in diff.items():
            mean_diff[key] += value
    
    print(mean_diff)

    # Compute means
    for key in mean_diff.keys():
        mean_diff[key] /= num_diffs

    return mean_diff


def print_comp(diff):
    print("Difference Metrics:")
    for key, value in diff.items():
        print(f"{key}: {value:.7f}")

In [15]:
sigma = 24
dossier_1 = "./images-blender/sigma/GT"
dossier_2 = f"./images-blender/sigma/{sigma}"

grp1 = []
grp2 = []

# Charger les 2 images de teste à comparer
for filename in os.listdir(dossier_1):
    if filename.endswith(".png") or filename.endswith(".jpg"):

        print(f"Loading {filename}...")

        img_path1 = os.path.join(dossier_1, filename)
        img_path2 = os.path.join(dossier_2, filename)

        img1 = Image.open(img_path1).convert("RGBA")
        img2 = Image.open(img_path2).convert("RGBA")

        arr1 = np.array(img1)
        arr2 = np.array(img2)

        grp1.append(arr1)
        grp2.append(arr2)

# Calculer la composition des deux images
comp = render_comp(grp1, grp2)

print(f"\nPour sigma = {sigma} :")
print_comp(comp)

Loading patatoide_0.png...
Loading patatoide_1.png...
Loading patatoide_2.png...
Loading patatoide_3.png...
Loading patatoide_4.png...
Loading patatoide_5.png...
Compared image 1/6
Compared image 2/6
Compared image 3/6
Compared image 4/6
Compared image 5/6
Compared image 6/6
{'NMAE': np.float32(2.9607506), 'NMSE': np.float32(1.1905296), 'ZNCC': np.float32(1.540011), 'PSNR': np.float32(59.63232), 'SSIM': np.float32(4.1976852)}

Pour sigma = 24 :
Difference Metrics:
NMAE: 0.4934584
NMSE: 0.1984216
ZNCC: 0.2566685
PSNR: 9.9387197
SSIM: 0.6996142
